# OpdsCredentialsModal E2E Accessibility Test

This notebook uses Playwright and axe-core to perform an end-to-end accessibility audit of the OpdsCredentialsModal component.

## 1. Set Up Test Environment

Install and import Playwright and axe-core for E2E and accessibility testing.

In [ ]:
# Install Playwright and axe-core if not already installed
!pip install playwright
!playwright install
!pip install axe-selenium-python

## 2. Render OpdsCredentialsModal Component

This cell launches the app and opens the OpdsCredentialsModal. Ensure your app has a button with `data-testid="open-opds-credentials-modal"` for test automation.

In [ ]:
from playwright.sync_api import sync_playwright
APP_URL = "http://localhost:3000"  # Update if your dev server runs elsewhere

def open_opds_credentials_modal():
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        page.goto(APP_URL)
        # Trigger the OpdsCredentialsModal (ensure a button with this test id exists in your app)
        page.click('[data-testid="open-opds-credentials-modal"]')
        return browser, page

browser, page = open_opds_credentials_modal()
print("OpdsCredentialsModal opened.")

## 3. Simulate User Interactions

This cell automates keyboard navigation, form filling, and other interactions to ensure the modal is accessible via keyboard and screen readers.

In [ ]:
# Tab through all focusable elements in the modal and print their accessible names
dialog = page.get_by_role('dialog')
focusable = dialog.query_selector_all('button, [href], input, select, textarea, [tabindex]:not([tabindex="-1"])')
assert len(focusable) > 0, "There should be at least one focusable element in the OpdsCredentialsModal"

for i, el in enumerate(focusable):
    if i == 0:
        # First element should be focused initially
        assert page.evaluate('document.activeElement === arguments[0]', el), f"Element {i} should be focused initially"
    else:
        page.keyboard.press('Tab')
        assert page.evaluate('document.activeElement === arguments[0]', el), f"Element {i} should be focused after Tab"
    print(f"Tab {i}:", el.inner_text())

## 4. Run Accessibility Audit

This cell integrates axe-core to scan the rendered modal for accessibility violations.

In [ ]:
# Inject and run axe-core for accessibility analysis
import requests
axe_url = "https://cdnjs.cloudflare.com/ajax/libs/axe-core/4.7.2/axe.min.js"
axe_script = requests.get(axe_url).text
page.add_script_tag(content=axe_script)
results = page.evaluate("async () => await axe.run(document.querySelector('[role=dialog]'))")
violations = results['violations']
if violations:
    print("Axe-core found accessibility violations:")
    for v in violations:
        print(f"- {v['id']}: {v['description']}")
else:
    print("No accessibility violations found by axe-core.")

## 5. Assert Accessibility Compliance

This cell checks the audit results and asserts that there are no critical accessibility issues. Results are output below.

In [ ]:
# Assert no critical accessibility violations
critical_violations = [v for v in violations if v['impact'] == 'critical']
if critical_violations:
    print("Critical accessibility violations found:")
    for v in critical_violations:
        print(f"- {v['id']}: {v['description']}")
    raise AssertionError("Critical accessibility violations detected!")
else:
    print("No critical accessibility violations detected. Accessibility test passed.")
browser.close()